# Information Theory

> 信息就是一个二进制序列，
>
> 也就是说不管信息量多大都全部编码成二进制，那我们就可以用二进制序列的长度来表示信息量的大小。

学习一下信息论

**reference** 
- https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/information-theory.html

## Self-information

> 一个二进制信息长度

$$
I(X) = - \log_2 (p)
$$

$$
I(\textrm{"0010"}) = - \log (p(\textrm{"0010"})) = - \log \left( \frac{1}{2^4} \right) = 4 \textrm{ bits}.
$$

In [1]:
import torch

def nansum(x):
    # Define nansum, as pytorch does not offer it inbuilt.
    return x[~torch.isnan(x)].sum()

def self_information(p):
    return -torch.log2(torch.tensor(p)).item()

self_information(1 / 64)

6.0

## Entropy

> 一堆二进制信息长度的期望值

### Definition

For any random variable $X$ that follows a probability distribution with a probability density function (p.d.f.) or a probability mass function (p.m.f.) $p(x)$, we measure the expected amount of information through entropy (or Shannon entropy):

$$
H(X) = \mathbb{E}_{x \sim P}[I(x)] = - \sum_{x \in \mathcal{X}} p(x) \log p(x).
$$

where
- $H(X)$ is the entropy of random variable $X$.
- $\mathcal{X}$ is the set of all possible outcomes of $X$.
- $p(x)$ is the p.d.f. or p.m.f. of $X$.
- $I(x) = - \log p(x)$ is the self-information of outcome $x$.
- $\mathbb{E}_{x \sim P}$ is the expectation of $x$ that follows the distribution $P$.
- $H(X)$ is the average amount(expected value) of information of $X$.

If $X$ is discrete, then

$$
H(X) = - \sum_i p_i \log p_i \textrm{, where } p_i = P(X_i).
$$

If $X$ is continuous, then

$$
H(X) = - \int_x p(x) \log p(x) \; dx.
$$

In [2]:
def entropy(p):
    entropy = - p * torch.log2(p)
    # Operator `nansum` will sum up the non-nan number
    out = nansum(entropy)
    return out

entropy(torch.tensor([0.1, 0.5, 0.1, 0.3]))

tensor(1.6855)

### Properties



- $H(X) \geq 0$ for any random variable $X$.
  - $H(X) = 0$ if and only if $X$ is deterministic. (After `log` operation, the value is 0 while all the probability is 1.)
> 信息长度肯定大于等于零，等于零的时候信息肯定是一串确定不变的东西，没任何含义。

- $H(X) \leq \log |\mathcal{X}|$ for any random variable $X$.
  - $H(X) = \log |\mathcal{X}|$ if and only if $X$ is a uniform distribution.
$$
H(X) \leq \log(k), \textrm{ with equality if and only if } p_i = \frac{1}{k}, \forall i.
$$

> 一堆二进制信息的长度期望值肯定小于等于二进制信息编码长度（信息样本空间的对数），等于的条件是样本空间中每种信息肯定是均匀分布。

- If $X \sim P$ and we try to encode $X$ with a code $f$, then the expected code length is $\mathbb{E}_{x \sim P}[l(f(x))] = \sum_{x \in \mathcal{X}} p(x) l(f(x))$.
  - $H(X)$ is the minimum expected code length among all possible codes.
  - $H(X)$ is the minimum number of bits required to encode $X$.

$$
H(X) = - E_{x \sim P} [\log p(x)] \leq  - E_{x \sim P} [\log q(x)], \textrm{ with equality if and only if } P = Q.
$$
> 说白了就是我想用一个函数来刻画这个信息，但最小期望信息长度肯定还是信息本身长度，函数的期望信息长度肯定大于等于信息本身，所以最小期望信息长度就是信息本身。

## Mutual Information

> 学习两种信息之间的关系，上面的信息都是单个信息的（单个分布），这里是两个信息之间的关系。（多个分布）

### Joint Entropy

> 两个信息共同想要表达的东西的长度期望值

> 一堆 两个信息的联合信息的长度期望值 (用一条信息表示两条信息)

$$
H(X, Y) =  -E_{(x, y) \sim P} [\log p_{X, Y}(x, y)].
$$

if $(X,Y) is a pair of discrete random variables, then

$$
H(X, Y) = - \sum_{x} \sum_{y} p_{X, Y}(x, y) \log p_{X, Y}(x, y).
$$

if $(X,Y) is a pair of continuous random variables, then

$$
H(X, Y) = - \int_{x, y} p_{X, Y}(x, y) \ \log p_{X, Y}(x, y) \;dx \;dy.
$$

#### Properties

- $H(X, Y) = H(X) + H(Y)$ if and only if $X$ and $Y$ are independent.

> 当这两条信息毫无关联的时候，联合信息长度肯定等于两个单独信息的信息长度之和。不然没法表示两条信息。

- $H(X, Y) = H(X) = H(Y)$ if and only if $X$ and $Y$ are identical.(means $X$ and $Y$ are the same random variable)

> 当这两条信息完全相同的时候，随便用其中一条就能表示两条了，所以联合信息长度等于单独信息长度。

- Always
$$
H(X), H(Y) \le H(X, Y) \le H(X) + H(Y).
$$
> 两个信息的联合信息长度肯定小于等于两个单独信息的信息长度之和，大于等于任一单独信息的信息长度。

In [3]:
def joint_entropy(p_xy):
    joint_ent = -p_xy * torch.log2(p_xy)
    # Operator `nansum` will sum up the non-nan number
    out = nansum(joint_ent)
    return out

joint_entropy(torch.tensor([[0.1, 0.5], [0.1, 0.3]]))

tensor(1.6855)

### Conditional Entropy

> 两个信息中，一条信息基于另一条信息想要表达的东西的长度期望值

> 一堆 一条信息的条件信息的长度期望值 (用一条信息表示另一条信息)

$$
H(Y|X) = E_{(x, y) \sim P} [ I(y|x) ] = -E_{(x, y) \sim P} [\log p_{Y|X}(y|x)]= - \sum_{x \in \mathcal{X}} \sum_{y \in \mathcal{Y}} p(x, y) \log p(y|x).
$$

if $(X,Y)$ is a pair of discrete random variables, then

$$
H(Y \mid X) = - \sum_{x} \sum_{y} p(x, y) \log p(y \mid x).
$$

if $(X,Y)$ is a pair of continuous random variables, then

$$
H(Y \mid X) = - \int_x \int_y p(x, y) \ \log p(y \mid x) \;dx \;dy.
$$

#### Properties

$$
H(Y \mid X) = H(X, Y) - H(X).
$$

> 一条信息的条件信息长度等于这条信息和另一条信息的联合信息长度减去另一条信息的信息长度。
> 
> 换种说法，`Y`信息基于`X`信息的信息量等于`Y`信息和`X`信息的联合信息量减去`X`信息的信息量。
> - `Y`信息基于`X`信息的信息量的意思就是`X`已经表达了的东西`Y`不需要再表达

In [4]:
def conditional_entropy(p_xy, p_x):
    p_y_given_x = p_xy/p_x
    cond_ent = -p_xy * torch.log2(p_y_given_x)
    # Operator `nansum` will sum up the non-nan number
    out = nansum(cond_ent)
    return out

conditional_entropy(
    torch.tensor([[0.1, 0.5], [0.2, 0.3]]),
    torch.tensor([0.2, 0.8])
)

tensor(0.8635)

### Mutual Information

> 两个信息的共享信息量

$$
I(X, Y) = H(X, Y) - H(Y \mid X) - H(X \mid Y).
$$

Expand it
$$
I(X, Y) = E_{x} E_{y} \left\{ p_{X, Y}(x, y) \log\frac{p_{X, Y}(x, y)}{p_X(x) p_Y(y)} \right\}.
$$

where
- $I(X, Y)$ is the mutual information between $X$ and $Y$.
- $H(X, Y)$ is the joint entropy of $X$ and $Y$.
- $H(X \mid Y)$ is the conditional entropy of $X$ given $Y$.
- $H(Y \mid X)$ is the conditional entropy of $Y$ given $X$.
- $p_{X, Y}(x, y)$ is the joint p.d.f. or p.m.f. of $X$ and $Y$.
- $p_X(x)$ is the p.d.f. or p.m.f. of $X$.
- $p_Y(y)$ is the p.d.f. or p.m.f. of $Y$.
- $E_{x}$ is the expectation of $x$.
- $E_{y}$ is the expectation of $y$.

Also, we have the following statements equivalent to the definition of mutual information:
- $I(X, Y) = H(X) - H(X \mid Y)$
- $I(X, Y) = H(Y) - H(Y \mid X)$
- $I(X, Y) = H(X) + H(Y) - H(X, Y)$ 

![mutual-information](image/mutual-information.svg)

In [5]:
def mutual_information(p_xy, p_x, p_y):
    p = p_xy / (p_x * p_y)
    mutual = p_xy * torch.log2(p)
    # Operator `nansum` will sum up the non-nan number
    out = nansum(mutual)
    return out

mutual_information(
    p_xy = torch.tensor([
        [0.1, 0.5], 
        [0.1, 0.3]
    ]),
    p_x = torch.tensor(
        [0.2, 0.8]
    ), 
    p_y = torch.tensor([
        [0.75, 0.25]
    ])
)

tensor(0.7195)

#### Properties

- $I(X, Y) \geq 0$ for any random variables $X$ and $Y$.
  - $I(X, Y) = 0$ if and only if $X$ and $Y$ are independent.

> 两个信息的共享信息量肯定大于等于零，等于零的时候两个信息毫无关联。

- $I(X, Y) = I(Y, X)$ for any random variables $X$ and $Y$.

> 两个信息的共享信息量和顺序无关。

- if $X$ is an invertible function of $Y$, then $I(X, Y) = H(X) = H(Y)$.

> 如果两个信息是可逆的，那么两个信息的共享信息量等于两个信息的信息量。

### Pointwise Mutual Information

> 记得在之前那个Expand

When we worked with entropy at the beginning of this chapter, we were able to provide an interpretation of $-\log(p_X(x))$
 as how surprised we were with the particular outcome. 

We may give a similar interpretation to the logarithmic term in the mutual information, which is often referred to as the pointwise mutual information:

$$
\textrm{pmi}(x, y) = \log\frac{p_{X, Y}(x, y)}{p_X(x) p_Y(y)}.
$$

- the denominator is $p_X(x) p_Y(y)$ which is the probability of the two outcomes were independent
- the numerator is $p_{X, Y}(x, y)$ which is the probability of the two outcomes were observed together

## Kullback-Leibler Divergence

> 两个信息量做差，然后求均值

> which provides a way to measure if two distributions are close together or not

> 在空间中我们可以用范数来刻画两个点之间的距离，但是在概率空间中也有很多方法刻画分布之间的距离，但信息论提供了很好的方法。

### Definition

$$
D_{\textrm{KL}}(P\|Q) = E_{x \sim P} \left[ \log \frac{p(x)}{q(x)} \right].
$$

where
- $D_{\textrm{KL}}(P\|Q)$ is the Kullback-Leibler divergence from $P$ to $Q$.
- $P$ and $Q$ are two distributions.
  - $P$ is the true distribution.
  - $Q$ is the estimated distribution. 
- $p(x)$ is the p.d.f. or p.m.f. of $P$.
- $q(x)$ is the p.d.f. or p.m.f. of $Q$.
- $E_{x \sim P}$ is the expectation of $x$ that follows the distribution $P$.
- $D_{\textrm{KL}}(P\|Q)$ is the expectation of $\log \frac{p(x)}{q(x)}$.

In [6]:
def kl_divergence(p, q):
    kl = p * torch.log2(p / q)
    out = nansum(kl)
    return out.abs().item()

### Properties

- non-negative
  - $D_{\textrm{KL}}(P\|Q) \geq 0$ for any distributions $P$ and $Q$.
  - $D_{\textrm{KL}}(P\|Q) = 0$ if and only if $P = Q$.
- non-symmetric
  - $D_{\textrm{KL}}(P\|Q) \neq D_{\textrm{KL}}(Q\|P)$
- close relationship between KL divergence and mutual information
$$
D_{\textrm{KL}}(P(X, Y)\|P(X)P(Y)) = E_{(x, y) \sim P} \left[ \log \frac{p(x, y)}{p(x)p(y)} \right] = I(X, Y)
$$

also equivalent to the following statements:
- $E_Y \{ D_{\textrm{KL}}(P(X \mid Y) \ \| \ P(X)) \}$
- $E_X \{ D_{\textrm{KL}}(P(Y \mid X) \ \| \ P(Y)) \}$

### Example 

>非对称性

In [7]:
torch.manual_seed(1)

tensor_len = 10000
p = torch.normal(0, 1, (tensor_len, ))
q1 = torch.normal(-1, 1, (tensor_len, ))
q2 = torch.normal(1, 1, (tensor_len, ))

p = torch.sort(p)[0]
q1 = torch.sort(q1)[0]
q2 = torch.sort(q2)[0]

In [8]:
kl_pq1 = kl_divergence(p, q1)
kl_pq2 = kl_divergence(p, q2)
similar_percentage = abs(kl_pq1 - kl_pq2) / ((kl_pq1 + kl_pq2) / 2) * 100

kl_pq1, kl_pq2, similar_percentage

(8582.0341796875, 8828.3095703125, 2.8290698237936858)

In [9]:
kl_q2p = kl_divergence(q2, p)
differ_percentage = abs(kl_q2p - kl_pq2) / ((kl_q2p + kl_pq2) / 2) * 100

kl_q2p, differ_percentage

(14130.125, 46.18621024399691)

## Cross Entropy

### Formal Definition

$$
\textrm{CE}(P, Q) = - E_{x \sim P} [\log(q(x))].
$$

where
- $\textrm{CE}(P, Q)$ is the cross entropy from $P$ to $Q$.
- $P$ and $Q$ are two distributions.
  - $P$ is the true distribution.
  - $Q$ is the estimated distribution.
- $q(x)$ is the p.d.f. or p.m.f. of $Q$.
- $E_{x \sim P}$ is the expectation of $x$ that follows the distribution $P$.

By definition, we have
$$
\textrm{CE}(P, Q) = H(P) + D_{\textrm{KL}}(P\|Q).
$$

In [14]:
def cross_entropy(y_hat, y):
    ce = -torch.log(y_hat[range(len(y_hat)), y])
    return ce.mean()

In [15]:
labels = torch.tensor([0, 2])
preds = torch.tensor([[0.3, 0.6, 0.1], [0.2, 0.3, 0.5]])

cross_entropy(preds, labels)

tensor(0.9486)

### Properties

Cross-entropy is always used to define a loss function in the optimization problem.

The following are equivalent:
- Maximizing predictive probability of 
$Q$ for distribution $P$,  ($E_{x \sim P} [\log (q(x))]$)
- Minimizing cross-entropy $\textrm{CE}(P, Q)$
- Minimizing KL divergence $D_{\textrm{KL}}(P\|Q)$

### `nn.NLLLoss`

$$
 \ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
        l_n = - w_{y_n} x_{n,y_n}, \quad
        w_{c} = \text{weight}[c] \cdot \mathbb{1}\{c \not= \text{ignore\_index}\},
$$

In [12]:
from torch import nn
nn.NLLLoss()(torch.log(preds), labels) # Negative Log Likelihood Loss

tensor(0.9486)

### `nn.CrossEntropyLoss`

> 这个函数它的默认第一个参数不是概率和为1的分布，而是网络最后一层的输出，然后自动执行`softmax`为你求一次概率。

$$
\ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
          l_n = - w_{y_n} \log \frac{\exp(x_{n,y_n})}{\sum_{c=1}^C \exp(x_{n,c})}
          \cdot \mathbb{1}\{y_n \not= \text{ignore\_index}\}
$$

In [13]:
nn.CrossEntropyLoss()(preds, labels)

tensor(1.0466)